# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [25]:
! pip install cartopy
! pip install geoviews
! pip install pyproj

In [32]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [33]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,fale old settlement,-9.3852,-171.2468,27.46,78,100,4.51,TK,1722885771
1,1,papatowai,-46.5619,169.4708,5.08,84,32,3.38,NZ,1722885591
2,2,blackmans bay,-43.0167,147.3167,10.41,74,100,1.79,AU,1722885773
3,3,qaqortoq,60.7167,-46.0333,9.45,69,100,1.65,GL,1722885774
4,4,kapa'a,22.0752,-159.3190,27.49,71,20,6.17,US,1722885519


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [34]:
# %%capture --no-display

# Configure the map plot
all_city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    alpha = 0.5,
    color = 'Country'
)

# Display the map
all_city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Country,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [36]:
# Narrow down cities that fit criteria and drop any results with null values
city_temp0 = city_data_df.loc[city_data_df['Max Temp']<27,:]
city_temp = city_temp0.loc[city_temp0['Max Temp']>21,:]
city_wind = city_temp.loc[city_temp['Wind Speed']<4.5,:]
city_cloud = city_wind.loc[city_wind['Cloudiness']==0,:]
city_cloud
# Drop any rows with null values
city_ideal = city_cloud.dropna()
# Display sample data
city_ideal.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,el granada,37.5027,-122.4694,23.60,73,0,2.57,US,1722885778
49,49,katsuura,35.1333,140.3000,24.54,92,0,2.85,JP,1722885825
97,97,fuxin,42.0156,121.6589,22.26,93,0,1.43,CN,1722885879
157,157,qutqashen,40.9814,47.8458,22.11,50,0,2.77,AZ,1722885949
198,198,san martin,-33.0810,-68.4681,23.30,10,0,2.88,AR,1722885950


### Step 3: Create a new DataFrame called `hotel_df`.

In [37]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_ideal.loc[:,['City','Country','Lat','Lng','Humidity']]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''
# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
6,el granada,US,37.5027,-122.4694,73,
49,katsuura,JP,35.1333,140.3000,92,
97,fuxin,CN,42.0156,121.6589,93,
157,qutqashen,AZ,40.9814,47.8458,50,
198,san martin,AR,-33.0810,-68.4681,10,
206,rudsar,IR,37.1376,50.2880,86,
223,arhribs,DZ,36.7936,4.3116,40,
329,cayiralan,TR,39.3028,35.6439,38,
331,hasaki,JP,35.7333,140.8333,90,
362,lincoln city,US,44.9582,-124.0179,77,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [38]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
el granada - nearest hotel: Beach House
katsuura - nearest hotel: 潮騒の宿
fuxin - nearest hotel: 凯文国际商务酒店
qutqashen - nearest hotel: Qəbələ Otel
san martin - nearest hotel: Hotel Lacava
rudsar - nearest hotel: هتل بهار
arhribs - nearest hotel: No hotel found
cayiralan - nearest hotel: No hotel found
hasaki - nearest hotel: Choshi Plaza Hotel
lincoln city - nearest hotel: Inn at Lincoln City
ketchikan - nearest hotel: Cape Fox Lodge
colonia - nearest hotel: Wasserturm Hotel Cologne
guelmim - nearest hotel: فندق الملتقى
sturgeon bay - nearest hotel: Stone Harbor
saint-francois - nearest hotel: Chez Lily
alta - nearest hotel: Scandic Alta
fernie - nearest hotel: Royal Hotel
bukama - nearest hotel: Mission catholique
conceicao do mato dentro - nearest hotel: Pires Pousada
prado - nearest hotel: Pousada Recanto do Prado
barcelos - nearest hotel: Hotel Dom Nuno
luau - nearest hotel: No hotel found
myrzakent - nearest hotel: No hotel found
nakanojomachi - nearest hotel: No 

,City,Country,Lat,Lng,Humidity,Hotel Name
6,el granada,US,37.5027,-122.4694,73,Beach House
49,katsuura,JP,35.1333,140.3000,92,潮騒の宿
97,fuxin,CN,42.0156,121.6589,93,凯文国际商务酒店
157,qutqashen,AZ,40.9814,47.8458,50,Qəbələ Otel
198,san martin,AR,-33.0810,-68.4681,10,Hotel Lacava
206,rudsar,IR,37.1376,50.2880,86,هتل بهار
223,arhribs,DZ,36.7936,4.3116,40,No hotel found
329,cayiralan,TR,39.3028,35.6439,38,No hotel found
331,hasaki,JP,35.7333,140.8333,90,Choshi Plaza Hotel
362,lincoln city,US,44.9582,-124.0179,77,Inn at Lincoln City


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [39]:
%%capture --no-display
# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    alpha =0.8,
    color = 'City',
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)